In [1]:
from langgraph.graph import StateGraph, END , START
from langchain_ollama.llms import OllamaLLM
from typing import TypedDict

In [2]:
model =  OllamaLLM(model="initium/law_model:latest")

In [3]:
class BlogState(TypedDict):
    title : str
    outline : str
    content : str

In [4]:
def create_outline(state: BlogState) -> BlogState:
    title = state['title']

    prompt = f'Generate a detailed outline for a blog on the topic {title}'

    outline = model.invoke({prompt})

    state['outline'] = outline 

    return state

    


In [6]:
def create_blog(state: BlogState) -> BlogState:
    title = state['title']

    outline = state['outline']

    prompt = f'Write a detailed blog on the title {title} using the following outline \n {outline}'

    content = model.invoke({prompt})

    state['content'] = content

    return state

    


In [9]:
graph = StateGraph(BlogState)

graph.add_node('create_outline', create_outline)

graph.add_node('create_blog', create_blog)


graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog',END)

workflow = graph.compile()

In [ ]:
initial_state = {'title':'Devops'}

final_state = workflow.invoke(initial_state)

